## Load FLORIS models and SOWFA DATA

Use the FLORIS tools to assign to each turbine in each SOWFA simulation, the number of turbines waking it, to identify different cases

In [1]:
import matplotlib.pyplot as plt
import floris.tools as wfct
import numpy as np
import pandas as pd
import copy
import seaborn as sns
import pickle
from scipy.optimize import minimize

In [2]:
df = pickle.load( open( "sowfa_01.p", "rb" ) )
fi_dict = pickle.load( open( "floris_models.p", "rb" ) )

In [3]:
# Just grab one FI for this part
(fi, color,marker, label) = fi_dict['g']

In [4]:
## Loop over the cases, assign the wake tables
wake_table = []
for i, row in df.iterrows():
    
     # Match the layout
    fi.reinitialize_flow_field(layout_array=[row.layout_x,row.layout_y])
    
    wake_table_temp = np.array(fi.floris.farm.turbine_map.number_of_wakes_iec(270.,return_turbines=False))
    num_free_stream = np.sum(wake_table_temp==0)
    
    if num_free_stream == 0:
        wake_table.append(np.nan)
    else:
        wake_table.append(wake_table_temp)


## Assign the wake tables back to df

In [5]:
df['wake_table'] = wake_table
df = df.dropna(subset=['wake_table'])

## Remake the index as a named hash

In [6]:
df = df.reset_index(drop=True)

In [7]:
df.head()

,Precursor,case_name,floris_TI,floris_U0,layout_x,layout_y,num_turbines,path,sim_length,sowfa_TI,sowfa_U0,yaw,power,ti,d_spacing,wake_table
0,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R-1_y-20,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 937.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[-20.0, 0.0]","[1697.0, 1445.7]","[0.09703534975568191, 0.16729992046512362]",5.0,"[0, 1]"
1,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R-1_y00,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 937.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[0.0, 0.0]","[1937.5, 1072.3]","[0.09297835821594881, 0.17988567028143654]",5.0,"[0, 1]"
2,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R-1_y20,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 937.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[20.0, 0.0]","[1688.3, 966.6]","[0.0853431860270954, 0.1327437170885774]",5.0,"[0, 1]"
3,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R00_y-20,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 1000.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[-20.0, 0.0]","[1699.3, 907.2]","[0.09688156249065837, 0.12228484151676836]",5.0,"[0, 1]"
4,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R00_y00,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 1000.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[0.0, 0.0]","[1938.0, 752.7]","[0.09285372321562335, 0.10368956548356832]",5.0,"[0, 1]"


In [8]:
df['casename'] = 'c_'
df['casename'] =  df['casename'] + df.index.values.astype(str)
df = df.set_index('casename')
df.head()

,Precursor,case_name,floris_TI,floris_U0,layout_x,layout_y,num_turbines,path,sim_length,sowfa_TI,sowfa_U0,yaw,power,ti,d_spacing,wake_table
casename,,,,,,,,,,,,,,,,
c_0,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R-1_y-20,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 937.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[-20.0, 0.0]","[1697.0, 1445.7]","[0.09703534975568191, 0.16729992046512362]",5.0,"[0, 1]"
c_1,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R-1_y00,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 937.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[0.0, 0.0]","[1937.5, 1072.3]","[0.09297835821594881, 0.17988567028143654]",5.0,"[0, 1]"
c_2,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R-1_y20,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 937.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[20.0, 0.0]","[1688.3, 966.6]","[0.0853431860270954, 0.1327437170885774]",5.0,"[0, 1]"
c_3,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R00_y-20,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 1000.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[-20.0, 0.0]","[1699.3, 907.2]","[0.09688156249065837, 0.12228484151676836]",5.0,"[0, 1]"
c_4,neutral_5kmx2kmx1km\n,c_08_highTI_D05_R00_y00,0.09,8.38,"(1000.0, 1630.0)","(1000.0, 1000.0)",2,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,8.0,"[0.0, 0.0]","[1938.0, 752.7]","[0.09285372321562335, 0.10368956548356832]",5.0,"[0, 1]"


In [9]:
# Save the updated dataframe
pickle.dump( df, open( "sowfa_03.p", "wb" ) )